In [9]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pickle
from mutation import *
from crossover import *
from rmp import *
from mfea import *
from task import *
from llm import *
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
# load_dotenv()

# GPT_API_KEY = os.getenv("GPT_API_KEY")

# llm = GPTModel(GPT_API_KEY, "gpt-3.5-turbo-0125", 0.7)

In [10]:
cec2017_tasks = get_10_tasks()

In [11]:
num_indis_per_task = 100
indi_len = 50
mutation = PolynomialMutation(5, 0.02)
crossover = SBXCrossover(mutation, eta=2)
# crossover = BLXCrossover()
rmp = AdaptiveRMPMatrix(rmp_pop_size=5, num_gen=3, pc=0.8, pm=0.1)
np.random.seed(0)
amfea = AMFEA(num_indis_per_task, indi_len, cec2017_tasks, crossover, mutation, rmp)

Initialization:
Task 0:
Best Fitness: 125473.96699307997
Mean Fitness: 166392.121167927

Task 1:
Best Fitness: 319204.0909081603
Mean Fitness: 503987.30720511527

Task 2:
Best Fitness: 346653.31463481847
Mean Fitness: 486737.256905658

Task 3:
Best Fitness: 41.0604113392313
Mean Fitness: 49.714217403069476

Task 4:
Best Fitness: 4058761661.357497
Mean Fitness: 6371677088.876898

Task 5:
Best Fitness: 21.482750101043788
Mean Fitness: 21.715487176250566

Task 6:
Best Fitness: 86.24595513767974
Mean Fitness: 99.8559653446189

Task 7:
Best Fitness: 18243.689210172182
Mean Fitness: 21153.768897608556

Task 8:
Best Fitness: 74.8950497645563
Mean Fitness: 122.64795239117853

Task 9:
Best Fitness: 88658.75615256254
Mean Fitness: 124389.07711105635



In [12]:
bfs, mfs = amfea.fit(num_gen=1000,monitor=True, monitor_rate=100, llm_rate=100)

Gen 0
Evaluation count: 0
Task 0, Best: 82192.89040593142, Avg: 138839.9283654692
Task 1, Best: 316014.31410111557, Avg: 430087.05940442055
Task 2, Best: 292894.69300041284, Avg: 400968.00669433194
Task 3, Best: 41.0604113392313, Avg: 45.937495812001195
Task 4, Best: 2027943133.4205256, Avg: 4741543962.937811
Task 5, Best: 21.33689326217712, Avg: 21.585582604853993
Task 6, Best: 80.48430943499493, Avg: 92.56483275812602
Task 7, Best: 17272.06912677073, Avg: 19688.46005266694
Task 8, Best: 73.94610143210085, Avg: 101.46741321674453
Task 9, Best: 75247.30020949453, Avg: 105568.75893567922
Time taken each gen: 0.1792 seconds

Creating strategy
Evaluating strategy
['Calculate the average fitness variance across all tasks and assign higher RMP to tasks with fitness variance below the average', 'Assign higher RMP to tasks with higher genetic diversity values to promote exploration in the search space', 'Consider the convergence rates of tasks and assign higher RMP to tasks with slower conver

<string>:13: RuntimeWarning: overflow encountered in exp


Evaluating strategy
['Prioritize tasks with higher genetic diversity by assigning higher random mating probabilities', 'Adjust random mating probabilities based on convergence rates to promote exploration in tasks with slower convergence', 'Fine-tune random mating probabilities based on fitness variance values to account for variability in fitness outcomes', 'Increase random mating probabilities for tasks with higher offspring success rates to capitalize on successful crossovers']
Creating code...
RMP function: import numpy as np

def get_rmp_matrix(task_count, diversity, convergence, fitness_variance, offspring_success_rate):
    rmp_matrix = np.zeros((task_count, task_count))
    
    for i in range(task_count):
        for j in range(i, task_count):
            rmp = (diversity[i] + diversity[j]) / 2
            rmp *= (1 + convergence[i] + convergence[j]) / 2
            rmp *= (1 + fitness_variance[i] + fitness_variance[j]) / 2
            rmp *= (1 + offspring_success_rate)
     

In [25]:
# num_tasks = len(tasks)
# fig, axes = plt.subplots(num_tasks, 2)
# fig.tight_layout()
# for i in range(num_tasks):
#     axes[i][0].plot(bfs[i])
#     axes[i][1].plot(mfs[i])